# (Day 1) Introduction to Natural Language Processing

Welcome to Day 1 of the Language Science Summer School 2022! 



In this short course, we will show you a glimpse of some of the cool stuff researchers have been doing with Natural Language Processing (NLP) systems. 
From story generation ([1](https://transformer.huggingface.co/)), to paraphrasing, or automatic reporting based on hundreds of documents ([2](https://www.askviable.com/)), or even helping customers filling their taxes ([3](https://www.keepertax.com/)), NLP research is assisting and automating decisions in various domains. 



## Agenda for today

We split this interactive notebook into 3 parts. In each part, we will ask you to interact with an NLP model and find interesting aspects.

- [15 min] Demo 1. Natural Language Inference
- [15 min] Demo 2. Question Generation
- [15 min] Demo 3. Story generation


## What to expect?

Before jumping into all the fun, we would like to briefly explain the format of these notebooks. Everyday, we will have an interactive notebook concerning the topics you learned about in that same day. These notebooks are meant to be interactive and fun :) The idea is that you will interact and use the techniques, you've discussed in class and try to find some advantages or limitations of these NLP systems. It turns out that, even though we are able to do amazing things using NLP, some models are still pretty wrong in some circumstances!

Don't worry! You won't do it alone! We're here to help and guide you!



# Part 1. Natural Language Inference

In this section, we will use a large language model to automatically determine whether a given *hypothesis* follows logically from a *premise*. In other words, given that you only know the *premise*, can we infer the *hypothesis* from it? Is it true? 


## Install and import libraries

For this part of the notebook, we will use HuggingFace Transformers library -- a popular framework amongst NLP researchers that provides out-of-the-box implementations of current state-of-the-art models.

Before using it, we need to install it.



In [ ]:
%%time
%%capture
!pip install sentencepiece
!pip install transformers

CPU times: user 102 ms, sys: 30.3 ms, total: 132 ms
Wall time: 14.4 s


In [ ]:
from torch.nn import functional as F
from transformers import BertTokenizer, BertForSequenceClassification

We will load a variant of the BERT language model that was fine tuned on a dataset with thousands of NLI examples.

In [ ]:
%%time
%%capture
model_name = "aloxatel/bert-base-mnli" 

# Load the tokenizer
tokenizer2 = BertTokenizer.from_pretrained(model_name)

# Load the model
model2 = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

CPU times: user 9.33 s, sys: 1.55 s, total: 10.9 s
Wall time: 15.2 s


In [ ]:
def get_nli_class(premise: str, hypothesis: str, with_score: bool=False):
  """
  Given a model, determine whether the hypothesis is inferred from the premise.
  """
  labels = ["contradiction", "entailment", "neutral"]

  encoded = tokenizer2.encode_plus(premise + hypothesis, max_length=100, padding="max_length", truncation=True, return_tensors="pt")
  outputs = model2(**encoded)

  outputs = F.softmax(outputs.logits, dim=-1)
  outputs = outputs.detach().numpy()
  
  # Retrieve the argmax
  label_ix = outputs.argmax()
  print("Premise:", premise, "\t | Hypothesis:", hypothesis)
  print("NLI label:", labels[label_ix])
  if with_score:
    print("NLI score:", round(outputs[0][label_ix], 3))

## Natural Language Inference (NLI)


In NLI, we are given two sentences: (1) a *premise* and (2) a *hypothesis*. The model must determine whether it is:
- **entailment**: if the *hypothesis* follows logically from *premise*;
- **contradiction**: if the *hypothesis* contradicts the *premise*;
- **neutral**: if the *hypothesis* has no effect on the *premise*.

A few examples are:

| Premise | Hypothesis | Label |
| ------- | ---------- | ----- |
| This church choir sings to the masses as they sing joyous songs from the book at a church. | The church is filled with song.  | Entailment |
| This church choir sings to the masses as they sing joyous songs from the book at a church. | A choir singing at a baseball game. | Contradiction |
| This church choir sings to the masses as they sing joyous songs from the book at a church.     | The church has cracks in the ceiling.        | Neutral |



In this part of the notebook, your task will be to use the method `get_nli_class` to find interesting model behaviors. You can do so by tweaking the `premise` or `hypothesis` parameters of the method. Here, is an example:


In [ ]:
premise = "The queen of England is bald."
hypothesis = "The queen is a man."

get_nli_class(premise, hypothesis, True)

Premise: The queen of England is bald. 	 | Hypothesis: The queen is a man.
NLI label: entailment
NLI score: 0.854


## Exercises

### [3 min] Warmup

In the next 2 to 3 minutes, play around with the to get familiar with the model. Try changing the inputs to the model, i.e., the *premise* and/or the *hypothesis*. 



In [ ]:
premise = ... # TODO: Write a premise
hypothesis = ... # TODO: Write an hypothesis

get_nli_class(premise=premise, hypothesis=hypothesis)


### [12 min] Exercise 

In groups, use the function `get_nli_class` and try to come up with examples where: 

- the model succeeds but humans would find hard.
- the model fails and humans would find hard.
- the model fails but humans would find easy.

Find **at least one example** for each of the bullet points above.

- Examples where **model succeeds and humans would find hard**

In [ ]:
premise = ... # TODO: Write a premise
hypothesis = ... # TODO: Write an hypothesis

get_nli_class(premise=premise, hypothesis=hypothesis)

- Examples where **model fails and humans would find hard**


In [ ]:
premise = ... # TODO: Write a premise
hypothesis = ... # TODO: Write an hypothesis

get_nli_class(premise=premise, hypothesis=hypothesis)

- Examples where **model fails but humans would find easy**


In [ ]:
premise = ... # TODO: Write a premise
hypothesis = ... # TODO: Write an hypothesis

get_nli_class(premise=premise, hypothesis=hypothesis)

# Part 2. Generating questions

In this section, we will resort to a different NLP model to generate questions automatically for us. Yup, that's right! This model is able to pose questions for our answers.


## Install and import libraries

In this section, we install and import the necessary frameworks. We also preload the models that will be used for the demo.

In [ ]:
%%time
%%capture
from transformers import AutoModelWithLMHead, AutoTokenizer

model_name = "mrm8488/t5-base-finetuned-question-generation-ap"

# Download tokenizer 
tokenizer3 = AutoTokenizer.from_pretrained(model_name)

# Download language model
model3 = AutoModelWithLMHead.from_pretrained(model_name)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


CPU times: user 20.6 s, sys: 3.68 s, total: 24.3 s
Wall time: 36.6 s


In [ ]:
def get_question(context, answer, max_length=64):
  input_text = f"answer: {answer}  context: {context} </s>"
  features = tokenizer3([input_text], return_tensors='pt')

  output = model3.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  question = tokenizer3.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  print(question)
  print()
  return question


## Question Generation

Let us now discuss how we can interact with our model. The model expects two different inputs: 
- **context**: two or three sentences that you will base the question and answers on. 
- **answer**: what you want the question to be about. 


Note that, for this particular model, it is better to specify *answers* that are contained in the provided *context*. 

In this part of the notebook, your task will be to use the method `generate_question` to find interesting model behaviors. You can do so by tweaking the `context` or `answer` parameters of the method. Here, is an example:

In [ ]:
%%time 
context = "Welcome to the Cognitive Science Summer School 2022."\
          "Catarina and Sameer are excited to tell you all about the"\
          "current research in NLP. Team up in groups of 4 and get ready"\
          "for five days of fun."
answer = "to tell you all about the NLP research"

get_question(answer, context)

question: What is the purpose of the Cognitive Science Summer School 2022?

CPU times: user 3.26 s, sys: 363 ms, total: 3.62 s
Wall time: 3.62 s


## Exercises

### [3 min] Warmup

In the next 2 to 3 minutes, play around with the to get familiar with the model. Try changing the inputs to the model, i.e., the *premise* and/or the *hypothesis*. 


In [ ]:
%%time 
context = ... # TODO: change the context
answer = ... # TODO: change the answer

get_question(answer, context)


### [12 min] Exercise 

In groups, use the function `get_question` and try to come up with examples where: 

- the model fails and humans would find hard.
- the model fails but humans would find easy.
- the model succeeds but humans could struggle.

Find **at least one example** for each of the bullet points above.

- the model fails and humans would also find hard.


In [ ]:
context = ... # TODO: Write context
answer = ... # TODO: Write an answer

get_question(answer, context)

- the model fails but humans would find easy.


In [ ]:
context = ... # TODO: Write context
answer = ... # TODO: Write an answer

get_question(answer, context)

- the model succeeds but humans could find hard.

In [ ]:
context = ... # TODO: Write context
answer = ... # TODO: Write an answer

get_question(answer, context)

# Part 3. Story generation 




In this part, we will ask you to interact with a large language model and use it to **generate different stories**. 
We will interact with a demo made available at [HuggingFace Spaces](https://huggingface.co/spaces/merve/GPT-2-story-gen).


## Story Generation 

In this demo, the model has a single input -- that we will tweak with! 
-  **prompt** is the text that you provide to the model and that will be used to generate a short story. Note, that this prompt will be the prefix of the generated story and the model will only append content to the story.

After defining your prompt and clicking *"Submit"*, the model will generate a story.


## [15 min] Exercise

In groups, use the story generation [demo](https://huggingface.co/spaces/merve/GPT-2-story-gen) described before to find examples where the output is interesting or wrong. In particular, try to come up with prompts that you wouldn't expect the model to perform well at. 

Copy this cell and fill it in with your own examples! 

```
# Example 1.
prompt:
category:
generated text: 
```


```
# Example 2.
prompt:
category:
generated text: 
```

# Concluding remarks

In today's notebook, we explored a few applications of different language models (including [BERT](https://arxiv.org/abs/1810.04805), [GPT2](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf), and [T5](https://github.com/google-research/text-to-text-transfer-transformer)).
Just as these demos, many other applications have been proposed and we challenge you to head over to [HuggingFace Spaces](https://huggingface.co/spaces?sort=likes) and have fun interacting with other applications, such as  [news-article summarization](https://huggingface.co/spaces/nickmuchi/article-text-summarizer), [rap lyrics generation](https://huggingface.co/spaces/Cropinky/gpt2-rap-songs), among others.
Optionally, if you're more into gaming 🎮, then [these AI-generated adventures](https://play.aidungeon.io/main/home) may be more exciting to explore!


As a last note, we would like to point you to an article that was fully generated by a GPT-base model. If you thought the generations in part 1 of this notebook were dull, then you must read this article titled [_A robot wrote this entire article. Are you scared yet, human?_](https://www.theguardian.com/commentisfree/2020/sep/08/robot-wrote-this-article-gpt-3).